In [1]:
import tqdm
import random
import pathlib
import itertools
import collections

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.losses import CategoricalCrossentropy

from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model
import tensorflow_addons as tfa

In [4]:
def setup():
    model_id = 'a2'
    resolution = 224
    batch_size=8
    num_frames=10
    tf.keras.backend.clear_session()
    use_positional_encoding = model_id in {'a3','a4','a5'}

    backbone = movinet.Movinet(model_id=model_id, conv_type ='2plus1d', se_type='2plus3d', activation='hard_swish', gating_activation='hard_sigmoid', use_positional_encoding=use_positional_encoding )
    backbone.trainable = True

    model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
    model.build([None, None, None, None, 3])

    checkpoint_callback = keras.callbacks.ModelCheckpoint(
            filepath='/content/drive/MyDrive/cp.ckpt3',
            monitor="val_loss",
            save_best_only=True,

        )

    def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
        model = movinet_model.MovinetClassifier(
        backbone=backbone,
        num_classes=num_classes)
        model.build([batch_size, num_frames, resolution, resolution, 3])

        return model

    model = build_classifier(batch_size, num_frames, resolution, backbone, 8)

    num_epochs = 1

    loss_obj = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    optimizer = tfa.optimizers.AdaBelief(learning_rate = 0.0007, weight_decay= 0.0001)
    model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])
    checkpoint = tf.train.Checkpoint(model)
    checkpoint.restore(r'C:\Users\Asus\Desktop\Fds project\Driver_Distraction\driver_distraction_demo\cp.ckpt1 (1)')
    return model

In [5]:
def play_demo(model,vid):

    options=['Safe Driving','Hair and Makeup', 'Infotainment System','Texting Left','Texting Right','Call left','Call Right','Drowsy']

    path=r"C:\Users\Asus\Desktop\Fds project\Driver_Distraction\driver_distraction_demo\demonew"
    path=os.path.join(path,vid)
    cap = cv2.VideoCapture(path)


    video_frames = np.zeros((1, 10, 224, 224, 3), dtype=np.float32)


    j=1
    while(j):
        for i in range(10):
            ret, frame = cap.read()
            if ret==False:
                j=0
                break
            frame = cv2.resize(frame, (224, 224))
            frame = frame.astype(np.float32)/255.0
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_frame[:, :, [0, 2]] = rgb_frame[:, :, [2, 0]]
            video_frames[0, i, :, :, :] = rgb_frame
        if (j==1):
            preds=model.predict(video_frames)
            prediction_p = tf.nn.softmax(preds)
        yhat = np.argmax(prediction_p,axis=1)
        if (j==1):
            for i in range(10):
                frame = video_frames[0, i, :, :, :]
                cv2.putText(frame, f"{options[int(yhat)]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.imshow('frame',frame)
                cv2.waitKey(150)

In [6]:
model=setup()

In [7]:
for i in os.listdir(r"C:\Users\Asus\Desktop\Fds project\Driver_Distraction\driver_distraction_demo\demonew"):
    play_demo(model,i)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 343ms/step
